<a href="https://colab.research.google.com/github/letsgo247/Deep-Learning-Study/blob/master/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#작은 데이터셋으로 강력한 이미지 분류 모델 설계하기

https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#used-in-the-notebooks_1

https://tykimos.github.io/2017/06/10/CNN_Data_Augmentation/

In [14]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='constant')   #'nearest':끝의 cell로 채우기, 'constant':걍 검은색, 'reflact':거울상, 'wrap':반복 채우기

img = load_img('/content/data/Osstem/김달원_#44,46.jpg')  # PIL 이미지
x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열

# 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
# 지정된 `preview/` 폴더에 저장합니다.
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='/content/data/Osstem/preview3', save_prefix='osstem', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # 이미지 20장을 생성하고 마칩니다